In [1]:
import pandas as pd
from pathlib import Path
import yfinance as yf
import os


Getting Nifty 50 constituents data

In [2]:
# Define the Nifty50 folder path
nifty50_dir = Path("/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/Nifty50")

# Find all nifty50_mcwb.csv files
csv_files = list(nifty50_dir.glob("*/nifty50_mcwb.csv"))
csv_files.sort()  # Sort for consistent ordering

In [3]:
def single_file(i):
    df = pd.read_csv(csv_files[i], header=2)
    df = df.iloc[:, 1:]  # Use .iloc to slice columns starting from the second column
    file_date = os.path.basename(os.path.dirname(csv_files[i]))[-5:]
    df.insert(0, 'Date', file_date)
    df['Date'] = pd.to_datetime(df['Date'] , format='%b%y') + pd.offsets.MonthEnd(0)
    df.dropna(inplace=True)
    return df



In [4]:
nifty50_cons = pd.concat([single_file(i) for i in range(len(csv_files))], ignore_index=True).sort_values(by='Date')
nifty50_cons

,Date,Security Symbol,Security Name,Industry,Equity Capital (In Rs.),Free Float Market Capitalisation (Rs. Crores),Weightage (%),Beta,R2,Volatility (%),Monthly Return,Avg. Impact Cost (%),Index Market Capitalisation (Rs. Crores)
0,2016-03-31,ACC,ACC Ltd.,CEMENT AND CEMENT PRODUCTS,1.881916e+09,13003.10,0.47,0.82,0.36,1.58,15.75,0.03,NaN
27,2016-03-31,INFY,Infosys Ltd.,COMPUTERS - SOFTWARE,1.148472e+10,243457.99,8.78,0.82,0.26,1.47,12.38,0.03,NaN
28,2016-03-31,KOTAKBANK,Kotak Mahindra Bank Ltd.,BANKS,9.166275e+09,69877.08,2.52,1.09,0.47,1.27,7.97,0.03,NaN
29,2016-03-31,LT,Larsen & Toubro Ltd.,ENGINEERING,1.862596e+09,99713.71,3.60,1.21,0.52,2.09,13.08,0.03,NaN
30,2016-03-31,LUPIN,Lupin Ltd.,PHARMACEUTICALS,9.010671e+08,35321.94,1.27,0.74,0.14,3.23,-15.69,0.03,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,2025-09-30,HINDALCO,Hindalco Industries Ltd.,ALUMINIUM,2.247227e+09,NaN,0.98,1.26,0.34,1.02,8.24,0.02,110049.75
972,2025-09-30,HINDUNILVR,Hindustan Unilever Ltd.,DIVERSIFIED,2.349591e+09,NaN,1.98,0.45,0.09,0.81,-5.47,0.01,222528.38
973,2025-09-30,ICICIBANK,ICICI Bank Ltd.,PRIVATE SECTOR BANK,1.428635e+10,NaN,8.52,0.89,0.45,0.63,-3.56,0.02,959257.48
963,2025-09-30,COALINDIA,Coal India Ltd.,INDUSTRIAL MINERALS,6.162728e+10,NaN,0.78,0.91,0.31,0.84,4.04,0.03,88309.73


In [5]:
# Ensure the directory exists
output_dir = Path("/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project")
# output_dir.mkdir(parents=True, exist_ok=True)

# Save the file
nifty50_cons.to_csv(output_dir / "nifty50_cons.csv", index=False)

In [6]:
nifty50_cons[['Date', 'Weightage (%)']].groupby('Date').sum()

,Weightage (%)
Date,
2016-03-31,99.96
2016-09-30,100.02
2017-03-31,100.00
2017-09-30,99.97
2018-03-31,100.03
2018-09-30,99.97
2019-03-31,100.01
2019-09-30,99.99
2020-03-31,100.02


Getting Daily Nifty 50 index values from 2021-2025

In [17]:
output_dir = Path("/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project")

ticker = "^NSEI"  # NIFTY 50 Index
start_date = "2015-01-01"
end_date = "2025-12-31"

print("Downloading NIFTY 50 index data from Yahoo Finance...")
df = yf.download(
    ticker,
    start=start_date,
    end=end_date,
    interval="1d",
    auto_adjust=False,
    progress=True,
)

if df.empty:
    raise RuntimeError("No data downloaded. Check ticker or date range.")

# Clean column names
df.reset_index(inplace=True)

# Remove MultiIndex from DataFrame columns
df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

output_file = "nifty50_index_prices_2015_2025.csv"

df.to_csv(output_dir / output_file, index=False)

print(f"Saved daily NIFTY 50 index prices to: {output_file}")
print(f"Rows: {len(df)}")

df




[*********************100%***********************]  1 of 1 completed

Saved daily NIFTY 50 index prices to: nifty50_index_prices_2015_2025.csv
Rows: 2707


,Date,Adj Close,Close,High,Low,Open,Volume
0,2015-01-02,8395.450195,8395.450195,8410.599609,8288.700195,8288.700195,101900
1,2015-01-05,8378.400391,8378.400391,8445.599609,8363.900391,8407.950195,118200
2,2015-01-06,8127.350098,8127.350098,8327.849609,8111.350098,8325.299805,172800
3,2015-01-07,8102.100098,8102.100098,8151.200195,8065.450195,8118.649902,164100
4,2015-01-08,8234.599609,8234.599609,8243.500000,8167.299805,8191.399902,143800
...,...,...,...,...,...,...,...
2702,2025-12-23,26177.150391,26177.150391,26233.550781,26119.050781,26205.199219,216600
2703,2025-12-24,26142.099609,26142.099609,26236.400391,26123.000000,26170.650391,188800
2704,2025-12-26,26042.300781,26042.300781,26144.199219,26008.599609,26121.250000,142200
2705,2025-12-29,25942.099609,25942.099609,26106.800781,25920.300781,26063.349609,234300


Get price for all unique securities to calcualte beta and vol with 1 year history from respective time period. In this case, we take from 2020 - 2025

In [18]:
base_dir = '/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/'
index_values_file = 'nifty50_index_prices_2015_2025.csv'
index_cons_file = 'nifty50_cons.csv'

In [19]:
index_cons = pd.read_csv(base_dir + index_cons_file, parse_dates=['Date'], usecols=['Date', 'Security Symbol'])
index_cons.insert(1, 'Index_name', 'Nifty 50')
index_cons

,Date,Index_name,Security Symbol
0,2016-03-31,Nifty 50,ACC
1,2016-03-31,Nifty 50,INFY
2,2016-03-31,Nifty 50,KOTAKBANK
3,2016-03-31,Nifty 50,LT
4,2016-03-31,Nifty 50,LUPIN
...,...,...,...
997,2025-09-30,Nifty 50,HINDALCO
998,2025-09-30,Nifty 50,HINDUNILVR
999,2025-09-30,Nifty 50,ICICIBANK
1000,2025-09-30,Nifty 50,COALINDIA


In [20]:
index_values = pd.read_csv(base_dir + index_values_file, usecols=['Date', 'Adj Close'], parse_dates=['Date'])
index_values

,Date,Adj Close
0,2015-01-02,8395.450195
1,2015-01-05,8378.400391
2,2015-01-06,8127.350098
3,2015-01-07,8102.100098
4,2015-01-08,8234.599609
...,...,...
2702,2025-12-23,26177.150391
2703,2025-12-24,26142.099609
2704,2025-12-26,26042.300781
2705,2025-12-29,25942.099609


In [21]:
unique_symbols = index_cons['Security Symbol'].unique()
unique_symbols_list = unique_symbols.tolist()
len(unique_symbols_list)

80

Get prices for all unique securites for beta and vol calculation. We will use 1 year history for each beta and vol calculation

In [22]:
start_date = "2015-01-01"
end_date = "2025-12-31"

In [23]:
failed_downloads = []
for symbol in unique_symbols_list:
    print(f"Downloading data for {symbol}...")
    try:
        stock_data = yf.download(symbol + '.NS', start=start_date, end=end_date)
        stock_data.reset_index(inplace=True)
        stock_data.columns = [col[0] if isinstance(col, tuple) else col for col in stock_data.columns]

        stock_data.to_csv(os.path.join(base_dir, 'price_history', f'{symbol}.csv'), index=False)
    except Exception as e:
        print(f"Error downloading data for {symbol}: {e}")
        failed_downloads.append(symbol)
        continue
print("Data download complete.")
if failed_downloads:
    print("Failed downloads for the following symbols:")
    for symbol in failed_downloads:
        print(symbol)

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

Data download complete.


In [24]:
print(failed_downloads)

[]


In [25]:
no_price_data_files = []
price_dir = Path(os.path.join(base_dir, 'price_history'))
for files in list(price_dir.glob("*csv")):
    df = pd.read_csv(os.path.join(base_dir, 'price_history', files))
    if len(df) == 0:
        print(f"File {files} is empty.")
        no_price_data_files.append(files)
no_price_data_files

File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/BRITANNIA.csv is empty.
File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/IBULHSGFIN.csv is empty.
File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/TATAMTRDVR.csv is empty.
File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/HDFC.csv is empty.
File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/ZOMATO.csv is empty.
File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/CAIRN.csv is empty.
File /Users/si

[PosixPath('/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/BRITANNIA.csv'),
 PosixPath('/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/IBULHSGFIN.csv'),
 PosixPath('/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/TATAMTRDVR.csv'),
 PosixPath('/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/HDFC.csv'),
 PosixPath('/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/ZOMATO.csv'),
 PosixPath('/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/CAIRN.csv'),
 PosixPath('/U

Deal with error files (manual intervention)

HDFC is HDFCBANK, and ZOMATO is now ETERNAL, which we have

TATAMOTORS is renamed as TMCV

In [26]:
symbol = 'TMCV'
single_stock = yf.download(symbol + '.NS', start=start_date, end=end_date)
single_stock.reset_index(inplace=True)
single_stock.columns = [col[0] if isinstance(col, tuple) else col for col in single_stock.columns]
single_stock
single_stock.to_csv(os.path.join(base_dir, 'price_history', f'{symbol}.csv'), index=False)

[*********************100%***********************]  1 of 1 completed


Single file for all price

In [27]:
price_data = pd.DataFrame()
price_dir = Path(os.path.join(base_dir, 'price_history'))
for files in list(price_dir.glob("*csv")):
    df = pd.read_csv(os.path.join(base_dir, 'price_history', files), usecols=['Date', 'Close', 'Volume'], parse_dates=['Date'])
    
    if len(df) == 0:
        print(f"File {files} is empty.")
    else:    
        df['Symbol'] = os.path.splitext(os.path.basename(files))[0]
        df.insert(0, 'Symbol', df.pop('Symbol'))
        price_data = pd.concat([price_data, df])

price_data.to_csv(os.path.join(base_dir, 'all_price_data.csv'), index=False)
price_data

File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/BRITANNIA.csv is empty.
File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/IBULHSGFIN.csv is empty.
File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/TATAMTRDVR.csv is empty.
File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/HDFC.csv is empty.
File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/ZOMATO.csv is empty.
File /Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/price_history/CAIRN.csv is empty.
File /Users/si

,Symbol,Date,Close,Volume
0,BHARTIARTL,2015-01-01,308.073578,2460188
1,BHARTIARTL,2015-01-02,309.898743,2795887
2,BHARTIARTL,2015-01-05,303.022461,1605267
3,BHARTIARTL,2015-01-06,300.645477,3916948
4,BHARTIARTL,2015-01-07,301.367035,4899318
...,...,...,...,...
2710,M&M,2025-12-23,3625.100098,1413628
2711,M&M,2025-12-24,3636.699951,842897
2712,M&M,2025-12-26,3623.100098,829910
2713,M&M,2025-12-29,3592.100098,1034692
